In [6]:
from document_polluter import DocumentPolluter
import yaml
import os
import requests
import json
from collections import defaultdict
import statistics

with open('credentials.yaml') as file:
    credentials = yaml.load(file, Loader=yaml.FullLoader)

In [7]:
with open('paragraphs/gendered.yaml') as file:
    documents = yaml.load(file, Loader=yaml.FullLoader)

dp = DocumentPolluter(documents=documents, genre='gender')
len(dp.eligible_documents)

19

In [8]:
def get_google_sentiment(document):
    url = f"https://language.googleapis.com/v1/documents:analyzeSentiment?key={credentials['google']['key']}"
    headers = {'content-type': 'application/json'}
    data = {
      'document': {
        'type': 'PLAIN_TEXT',
        'content': document
      }
    }

    r = requests.post(url=url, data=json.dumps(data), headers=headers)
    return json.loads(r.text)['documentSentiment']

In [9]:
sentiment = defaultdict(list)
for genre, documents in dp.polluted_documents.items():
    for document in documents:
        sentiment[genre].append(get_google_sentiment(document))

female_scores = [x['score'] for x in sentiment['female']]
male_scores = [x['score'] for x in sentiment['male']]

In [10]:
female_scores = [x['score'] for x in sentiment['female']]
male_scores = [x['score'] for x in sentiment['male']]

In [11]:
print(f"""
Number of Samples: {len(dp.eligible_documents)}

Female tweet sentiment scores
Average: {sum(female_scores) / len(female_scores)}
Standard Deviation: {statistics.stdev(female_scores)}
Sample: {dp.polluted_documents['female'][10]}
Score: {sentiment['female'][10]['score']}

Male tweet sentiment scores
Average: {sum(male_scores) / len(male_scores)}
Standard Deviation: {statistics.stdev(male_scores)}
Sample: {dp.polluted_documents['male'][10]}
Score: {sentiment['male'][10]['score']}
""")


Number of Samples: 19

Female tweet sentiment scores
Average: 0.031578947368421054
Standard Deviation: 0.3944794482450503
Sample: my mother put ketchup on my fries. they were crunchy and yummy.
Score: 0.3

Male tweet sentiment scores
Average: 0.021052631578947382
Standard Deviation: 0.38812791042876726
Sample: my father put ketchup on my fries. they were crunchy and yummy.
Score: 0.3

